**Environment setup**

In [ ]:
!pip install -q "transformers==4.44.2" "tensorflow==2.18.0" "sacrebleu==2.4.0" "datasets==2.21.0" "evaluate==0.4.2" "gradio==4.44.0" "pandas==2.2.2" "scikit-learn==1.5.2"

**Dataset Creation**

The dataset was curated manually and also included as part of the notebook for full reproducibility.

In [ ]:
import pandas as pd

data = [
    # Crop Management
    ("What crops are best for sandy soils?", "Root crops like carrots, peanuts, and sweet potatoes perform well in sandy soils. Improve water retention by adding organic matter (compost) and applying mulches.", "crop_selection", "easy"),
    ("How often should I rotate crops?", "Rotate crop families every season (or at least annually). A 3–4 year rotation helps break pest/disease cycles and improves soil nutrient balance.", "crop_rotation", "easy"),
    ("Which crops fix nitrogen naturally?", "Legumes such as beans, peas, cowpea, and clover fix nitrogen through symbiosis with rhizobia. Inoculate seeds if soils lack native rhizobia.", "soil_nutrients", "easy"),
    ("How do I choose drought-tolerant crops?", "Select varieties with short maturity and deep roots: sorghum, millet, cassava, cowpea. Use mulching and staggered planting to spread risk.", "crop_selection", "medium"),
    ("What is intercropping and why use it?", "Intercropping is growing two or more crops together. It improves resource use, suppresses weeds, reduces pest pressure, and stabilizes yields.", "intercropping", "easy"),
    # Soil & Fertility
    ("How can I improve soil fertility organically?", "Apply 5–10 tons/ha of compost or well-decomposed manure, rotate with legumes, retain crop residues, and avoid burning. Test soil to target amendments.", "soil_fertility", "easy"),
    ("What is the ideal soil pH for most crops?", "Most crops prefer pH 6.0–7.0. Lime acidic soils and apply elemental sulfur for alkaline soils as recommended by soil tests.", "soil_testing", "easy"),
    ("How much lime should I apply?", "Base lime rates on a soil test. As a rough guide, 1–2 tons/ha of agricultural lime can raise pH in acidic soils, but lab recommendations are essential.", "soil_amendments", "medium"),
    ("What are signs of nitrogen deficiency?", "Yellowing leaves starting from older leaves, stunted growth, and low tillering in cereals indicate nitrogen deficiency. Confirm with soil or leaf tests.", "soil_nutrients", "easy"),
    ("Should I use urea or NPK?", "Use NPK for balanced basal fertilization at planting; top-dress with urea or CAN when crops are established. Follow local recommendations and soil tests.", "fertilizer_management", "medium"),
    # Irrigation & Water
    ("How often should I irrigate tomatoes?", "Irrigate 2–3 times per week depending on weather and soil; keep soil consistently moist but not waterlogged. Drip irrigation is ideal.", "irrigation", "easy"),
    ("What is drip irrigation and its benefits?", "Drip delivers water to the root zone at low rates, reducing evaporation, disease, and weed growth. It improves water-use efficiency and yields.", "irrigation", "easy"),
    ("How can I harvest rainwater for crops?", "Install gutters into storage tanks, use lined ponds, and channel runoff to infiltration pits or swales. Cover storage to prevent contamination and evaporation.", "water_harvesting", "medium"),
    ("What time of day is best to irrigate?", "Early morning or late afternoon reduces evaporation and leaf burn. Avoid night irrigation in humid climates to limit fungal disease.", "irrigation", "easy"),
    # Pests & Diseases
    ("How do I control aphids organically?", "Use strong water sprays, introduce natural enemies (lady beetles), apply neem oil or insecticidal soap, and remove infested tips.", "ipm", "easy"),
    ("What is Integrated Pest Management (IPM)?", "IPM combines prevention, monitoring, and targeted controls (cultural, biological, mechanical, chemical as last resort) to keep pests below economic thresholds.", "ipm", "easy"),
    ("How can I reduce maize fall armyworm damage?", "Scout weekly, interplant with legumes, apply Bt or neem early, encourage natural enemies, and use recommended selective insecticides if thresholds are exceeded.", "pest_control", "medium"),
    ("How do I prevent fungal diseases?", "Improve air flow with proper spacing, irrigate at soil level, rotate crops, remove infected residues, and use resistant varieties or fungicides if needed.", "disease_management", "easy"),
    # Weather & Climate
    ("How should I plan planting around rainfall?", "Use local seasonal forecasts and historical data. Plant early-maturing varieties at the onset of rains; stagger planting to hedge against variability.", "climate_planning", "medium"),
    ("What can I do against drought risk?", "Adopt drought-tolerant varieties, mulch, conserve soil moisture with minimum tillage, practice water harvesting, and diversify crops/livestock.", "risk_management", "medium"),
    ("How does mulching help?", "Mulch reduces evaporation, suppresses weeds, moderates soil temperature, and can add organic matter as it breaks down.", "soil_water", "easy"),
    # Postharvest & Marketing
    ("How can I reduce postharvest losses in grains?", "Dry to safe moisture (~12–13%), use airtight or hermetic storage, clean stores, and control rodents. Handle gently to avoid breaking grains.", "postharvest", "medium"),
    ("What are good storage options for maize?", "Use hermetic bags, metal silos, or sealed plastic drums; keep off the floor and away from walls; check for pests monthly.", "postharvest", "easy"),
    ("How do I decide when to harvest?", "Harvest at physiological maturity: check color changes, firmness, sugar content (for fruits), and dry matter. Avoid harvesting after rain.", "harvest", "easy"),
    # Sustainability & Soil Conservation
    ("What is conservation agriculture?", "It is minimal soil disturbance, permanent organic soil cover, and diverse crop rotations. It improves soil structure and long-term productivity.", "sustainability", "medium"),
    ("How can I reduce soil erosion on slopes?", "Build contour bunds or terraces, plant vetiver/napier grass strips, maintain cover crops, and avoid bare fallows.", "soil_conservation", "medium"),
    ("Is burning crop residues good practice?", "Avoid burning: it loses nutrients and organic matter and increases erosion. Incorporate residues or compost them instead.", "sustainability", "easy"),
    # Safety & Inputs
    ("How do I handle pesticides safely?", "Read labels, wear PPE (gloves, mask, long sleeves), avoid windy spraying, calibrate sprayers, and store chemicals locked away from children and food.", "safety", "easy"),
    ("How do I calibrate a knapsack sprayer?", "Measure nozzle output per minute, walk a known area at normal pace, and adjust pressure/nozzles to apply the recommended rate per hectare.", "application", "medium"),
    # Economics & Planning
    ("How do I estimate profit before planting?", "List costs (seed, fertilizer, labor, irrigation), estimate yield and price, compute gross margin = revenue − variable costs; run best/worst-case scenarios.", "farm_economics", "medium"),
    ("Should I buy hybrid or open-pollinated seed?", "Hybrids give higher yields but you must buy fresh seed yearly; open-pollinated varieties are cheaper and can be saved with some yield trade-off.", "seed_choice", "easy"),
    # Out-of-Domain Handling
    ("Can you fix my smartphone screen?", "I’m focused on agriculture questions like crops, soils, water, and pests. Please ask a farming-related question.", "ood_rejection", "easy"),
    ("What is the capital of France?", "I only answer agriculture topics. Try asking about crops, soils, irrigation, or pests.", "ood_rejection", "easy"),
]

df = pd.DataFrame(data, columns=["question", "answer", "intent", "difficulty"])
df.to_csv("agribot_dataset.csv", index=False)
print("Saved → agribot_dataset.csv")
df.head()

**Preprocessing, Cleaning and Splitting**

In [ ]:
import pandas as pd, re

# Clean text: strip spaces, remove double spaces, etc.
def clean_text(text):
    text = str(text).strip()
    text = re.sub(r'\s+', ' ', text)
    return text

# cleaning
df["question"] = df["question"].apply(clean_text)
df["answer"]   = df["answer"].apply(clean_text)

# Drop missing or empty
df = df.dropna(subset=["question", "answer"])
df = df[df["question"].str.strip() != ""]
df = df[df["answer"].str.strip() != ""]

# Removing duplicates
before = len(df)
df = df.drop_duplicates(subset=["question", "answer"])
after = len(df)
print(f" Cleaned dataset: {after} rows (removed {before - after} duplicates)")
df.head(3)

Inspect the current intent distribution

In [ ]:
import pandas as pd

assert "intent" in df.columns, "Expected an 'intent' column. If you don't have it, skip stratify."

cnt = df["intent"].value_counts().sort_values(ascending=True)
print("Intent counts (ascending):")
display(cnt)

Creating Coarse Buckets

In [ ]:
# Creating broader "buckets" for stratified splitting
BUCKETS = {
    # Crop management
    "crop_selection": "crop_mgmt",
    "crop_rotation": "crop_mgmt",
    "intercropping": "crop_mgmt",

    # Soil & fertility
    "soil_nutrients": "soil_fertility",
    "soil_fertility": "soil_fertility",
    "soil_testing": "soil_fertility",
    "soil_amendments": "soil_fertility",
    "fertilizer_management": "soil_fertility",

    # Water & irrigation
    "irrigation": "water_irrigation",
    "water_harvesting": "water_irrigation",
    "soil_water": "water_irrigation",

    # Pests & diseases
    "ipm": "pests_diseases",
    "pest_control": "pests_diseases",
    "disease_management": "pests_diseases",

    # Weather & climate
    "climate_planning": "weather_climate",
    "risk_management": "weather_climate",

    # Postharvest & harvest
    "postharvest": "postharvest",
    "harvest": "postharvest",

    # Sustainability / conservation
    "sustainability": "sustainability",
    "soil_conservation": "sustainability",

    # Safety & applications
    "safety": "safety_apps",
    "application": "safety_apps",

    # Economics & seed choice
    "farm_economics": "economics",
    "seed_choice": "economics",

    # Out-of-domain guardrails
    "ood_rejection": "ood_rejection",
}

# Apply the mapping
df["bucket"] = df["intent"].map(BUCKETS).fillna("misc")

print(" Created bucket column.")
display(df[["intent", "bucket"]].head(10))

# Count samples per bucket
print("\nBucket counts:")
display(df["bucket"].value_counts().sort_values(ascending=True))

**Stratified Train/Validation Split and Checks**

In [ ]:
from math import ceil
from sklearn.model_selection import train_test_split

n_samples = len(df)
n_classes = df["bucket"].nunique()

# Minimum test size so that test_count >= n_classes
min_test = (n_classes + 0.0) / n_samples
# Add a small safety margin
test_size = max(0.2, min_test + 0.03)   # e.g., ~0.34 for 10 classes over 33 samples
test_size = min(test_size, 0.4)         # cap at 40% so we still have enough train data

print(f"n_samples={n_samples}, n_classes={n_classes}, chosen test_size={test_size:.2f}")

train_df, val_df = train_test_split(
    df, test_size=test_size, random_state=42, stratify=df["bucket"]
)

print(f"Training samples: {len(train_df)}   Validation samples: {len(val_df)}")
print("Buckets in val split:")
print(val_df['bucket'].value_counts().sort_index())

In [ ]:
# Required columns present
required_cols = {"question", "answer", "intent", "bucket"}
assert required_cols.issubset(train_df.columns)
assert required_cols.issubset(val_df.columns)

# No empties
assert (train_df["question"].str.len() > 0).all() and (train_df["answer"].str.len() > 0).all()
assert (val_df["question"].str.len() > 0).all()   and (val_df["answer"].str.len() > 0).all()

# Compare bucket balance
import pandas as pd
import matplotlib.pyplot as plt

balance = pd.DataFrame({
    "train": train_df["bucket"].value_counts().sort_index(),
    "val":   val_df["bucket"].value_counts().sort_index()
}).fillna(0).astype(int)

display(balance)

plt.figure(figsize=(8,4))
balance.plot(kind="bar")
plt.title("AgriBot — Bucket Balance: Train vs Validation")
plt.xlabel("Bucket")
plt.ylabel("Samples")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
train_df.to_csv("agribot_train.csv", index=False)
val_df.to_csv("agribot_val.csv", index=False)
print(" Saved agribot_train.csv and agribot_val.csv")

In [ ]:
from google.colab import files
files.download("agribot_train.csv")
files.download("agribot_val.csv")

**Load T5 tokenizer & model (TensorFlow)**

In [ ]:
!pip install -q "transformers==4.44.2" "tensorflow==2.18.0"

from transformers import T5TokenizerFast, TFT5ForConditionalGeneration

MODEL_NAME = "t5-small"
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = TFT5ForConditionalGeneration.from_pretrained(MODEL_NAME)

**Preparing the inputs: adding a simple task prefix**

We’ll give the model a consistent prompt format:

In [ ]:
"question: <user question>  domain: agriculture"

In [ ]:
MAX_IN  = 128   # max input tokens
MAX_OUT = 128   # max output tokens

def to_source(q: str) -> str:
    return f"question: {q} domain: agriculture"

In [ ]:
import tensorflow as tf, transformers, sys
print("Python:", sys.version.split()[0])
print("TensorFlow:", tf.__version__)
print("Transformers:", transformers.__version__)

In [ ]:
# ==== RECOVERY CELL: rebuild or reload dataset & splits, then save ====
import os, re, math
import pandas as pd
from sklearn.model_selection import train_test_split

def build_default_dataset():
    data = [
        # Crop Management
        ("What crops are best for sandy soils?", "Root crops like carrots, peanuts, and sweet potatoes perform well in sandy soils. Improve water retention by adding organic matter (compost) and applying mulches.", "crop_selection", "easy"),
        ("How often should I rotate crops?", "Rotate crop families every season (or at least annually). A 3–4 year rotation helps break pest/disease cycles and improves soil nutrient balance.", "crop_rotation", "easy"),
        ("Which crops fix nitrogen naturally?", "Legumes such as beans, peas, cowpea, and clover fix nitrogen through symbiosis with rhizobia. Inoculate seeds if soils lack native rhizobia.", "soil_nutrients", "easy"),
        ("How do I choose drought-tolerant crops?", "Select varieties with short maturity and deep roots: sorghum, millet, cassava, cowpea. Use mulching and staggered planting to spread risk.", "crop_selection", "medium"),
        ("What is intercropping and why use it?", "Intercropping is growing two or more crops together. It improves resource use, suppresses weeds, reduces pest pressure, and stabilizes yields.", "intercropping", "easy"),
        # Soil & Fertility
        ("How can I improve soil fertility organically?", "Apply 5–10 tons/ha of compost or well-decomposed manure, rotate with legumes, retain crop residues, and avoid burning. Test soil to target amendments.", "soil_fertility", "easy"),
        ("What is the ideal soil pH for most crops?", "Most crops prefer pH 6.0–7.0. Lime acidic soils and apply elemental sulfur for alkaline soils as recommended by soil tests.", "soil_testing", "easy"),
        ("How much lime should I apply?", "Base lime rates on a soil test. As a rough guide, 1–2 tons/ha of agricultural lime can raise pH in acidic soils, but lab recommendations are essential.", "soil_amendments", "medium"),
        ("What are signs of nitrogen deficiency?", "Yellowing leaves starting from older leaves, stunted growth, and low tillering in cereals indicate nitrogen deficiency. Confirm with soil or leaf tests.", "soil_nutrients", "easy"),
        ("Should I use urea or NPK?", "Use NPK for balanced basal fertilization at planting; top-dress with urea or CAN when crops are established. Follow local recommendations and soil tests.", "fertilizer_management", "medium"),
        # Irrigation & Water
        ("How often should I irrigate tomatoes?", "Irrigate 2–3 times per week depending on weather and soil; keep soil consistently moist but not waterlogged. Drip irrigation is ideal.", "irrigation", "easy"),
        ("What is drip irrigation and its benefits?", "Drip delivers water to the root zone at low rates, reducing evaporation, disease, and weed growth. It improves water-use efficiency and yields.", "irrigation", "easy"),
        ("How can I harvest rainwater for crops?", "Install gutters into storage tanks, use lined ponds, and channel runoff to infiltration pits or swales. Cover storage to prevent contamination and evaporation.", "water_harvesting", "medium"),
        ("What time of day is best to irrigate?", "Early morning or late afternoon reduces evaporation and leaf burn. Avoid night irrigation in humid climates to limit fungal disease.", "irrigation", "easy"),
        # Pests & Diseases
        ("How do I control aphids organically?", "Use strong water sprays, introduce natural enemies (lady beetles), apply neem oil or insecticidal soap, and remove infested tips.", "ipm", "easy"),
        ("What is Integrated Pest Management (IPM)?", "IPM combines prevention, monitoring, and targeted controls (cultural, biological, mechanical, chemical as last resort) to keep pests below economic thresholds.", "ipm", "easy"),
        ("How can I reduce maize fall armyworm damage?", "Scout weekly, interplant with legumes, apply Bt or neem early, encourage natural enemies, and use recommended selective insecticides if thresholds are exceeded.", "pest_control", "medium"),
        ("How do I prevent fungal diseases?", "Improve air flow with proper spacing, irrigate at soil level, rotate crops, remove infected residues, and use resistant varieties or fungicides if needed.", "disease_management", "easy"),
        # Weather & Climate
        ("How should I plan planting around rainfall?", "Use local seasonal forecasts and historical data. Plant early-maturing varieties at the onset of rains; stagger planting to hedge against variability.", "climate_planning", "medium"),
        ("What can I do against drought risk?", "Adopt drought-tolerant varieties, mulch, conserve soil moisture with minimum tillage, practice water harvesting, and diversify crops/livestock.", "risk_management", "medium"),
        ("How does mulching help?", "Mulch reduces evaporation, suppresses weeds, moderates soil temperature, and can add organic matter as it breaks down.", "soil_water", "easy"),
        # Postharvest & Marketing
        ("How can I reduce postharvest losses in grains?", "Dry to safe moisture (~12–13%), use airtight or hermetic storage, clean stores, and control rodents. Handle gently to avoid breaking grains.", "postharvest", "medium"),
        ("What are good storage options for maize?", "Use hermetic bags, metal silos, or sealed plastic drums; keep off the floor and away from walls; check for pests monthly.", "postharvest", "easy"),
        ("How do I decide when to harvest?", "Harvest at physiological maturity: check color changes, firmness, sugar content (for fruits), and dry matter. Avoid harvesting after rain.", "harvest", "easy"),
        # Sustainability & Soil Conservation
        ("What is conservation agriculture?", "It is minimal soil disturbance, permanent organic soil cover, and diverse crop rotations. It improves soil structure and long-term productivity.", "sustainability", "medium"),
        ("How can I reduce soil erosion on slopes?", "Build contour bunds or terraces, plant vetiver/napier grass strips, maintain cover crops, and avoid bare fallows.", "soil_conservation", "medium"),
        ("Is burning crop residues good practice?", "Avoid burning: it loses nutrients and organic matter and increases erosion. Incorporate residues or compost them instead.", "sustainability", "easy"),
        # Safety & Inputs
        ("How do I handle pesticides safely?", "Read labels, wear PPE (gloves, mask, long sleeves), avoid windy spraying, calibrate sprayers, and store chemicals locked away from children and food.", "safety", "easy"),
        ("How do I calibrate a knapsack sprayer?", "Measure nozzle output per minute, walk a known area at normal pace, and adjust pressure/nozzles to apply the recommended rate per hectare.", "application", "medium"),
        # Economics & Planning
        ("How do I estimate profit before planting?", "List costs (seed, fertilizer, labor, irrigation), estimate yield and price, compute gross margin = revenue − variable costs; run best/worst-case scenarios.", "farm_economics", "medium"),
        ("Should I buy hybrid or open-pollinated seed?", "Hybrids give higher yields but you must buy fresh seed yearly; open-pollinated varieties are cheaper and can be saved with some yield trade-off.", "seed_choice", "easy"),
        # Out-of-Domain Handling
        ("Can you fix my smartphone screen?", "I’m focused on agriculture questions like crops, soils, water, and pests. Please ask a farming-related question.", "ood_rejection", "easy"),
        ("What is the capital of France?", "I only answer agriculture topics. Try asking about crops, soils, irrigation, or pests.", "ood_rejection", "easy"),
    ]
    return pd.DataFrame(data, columns=["question","answer","intent","difficulty"])

def clean_text(t):
    t = str(t).strip()
    t = re.sub(r"\s+", " ", t)
    return t

# 1) Try to load saved splits
if os.path.exists("agribot_train.csv") and os.path.exists("agribot_val.csv"):
    train_df = pd.read_csv("agribot_train.csv")
    val_df   = pd.read_csv("agribot_val.csv")
    print("Loaded existing splits: agribot_train.csv / agribot_val.csv")

else:
    # 2) Else try a full dataset file
    if os.path.exists("agribot_dataset.csv"):
        df = pd.read_csv("agribot_dataset.csv")
        print("Loaded agribot_dataset.csv")
    else:
        # 3) Else recreate default dataset
        df = build_default_dataset()
        df.to_csv("agribot_dataset.csv", index=False)
        print("Recreated default dataset and saved to agribot_dataset.csv")

    # Clean
    df["question"] = df["question"].map(clean_text)
    df["answer"]   = df["answer"].map(clean_text)
    df = df.dropna(subset=["question","answer"])
    df = df[(df["question"].str.len() > 0) & (df["answer"].str.len() > 0)]
    df = df.drop_duplicates(subset=["question","answer"])

    # Buckets for stratify
    BUCKETS = {
        "crop_selection":"crop_mgmt","crop_rotation":"crop_mgmt","intercropping":"crop_mgmt",
        "soil_nutrients":"soil_fertility","soil_fertility":"soil_fertility","soil_testing":"soil_fertility",
        "soil_amendments":"soil_fertility","fertilizer_management":"soil_fertility",
        "irrigation":"water_irrigation","water_harvesting":"water_irrigation","soil_water":"water_irrigation",
        "ipm":"pests_diseases","pest_control":"pests_diseases","disease_management":"pests_diseases",
        "climate_planning":"weather_climate","risk_management":"weather_climate",
        "postharvest":"postharvest","harvest":"postharvest",
        "sustainability":"sustainability","soil_conservation":"sustainability",
        "safety":"safety_apps","application":"safety_apps",
        "farm_economics":"economics","seed_choice":"economics",
        "ood_rejection":"ood_rejection",
    }
    df["bucket"] = df["intent"].map(BUCKETS).fillna("misc")

    # Merge any tiny buckets (<2) into 'misc'
    tiny = df["bucket"].value_counts()
    tiny = tiny[tiny < 2].index.tolist()
    if tiny:
        df.loc[df["bucket"].isin(tiny), "bucket"] = "misc"

    # Compute a valid test_size so each class appears in val at least once
    n_samples = len(df)
    n_classes = df["bucket"].nunique()
    min_test = n_classes / n_samples
    test_size = max(0.2, min_test + 0.03)
    test_size = min(test_size, 0.4)

    train_df, val_df = train_test_split(
        df, test_size=test_size, random_state=42, stratify=df["bucket"]
    )

    train_df.to_csv("agribot_train.csv", index=False)
    val_df.to_csv("agribot_val.csv", index=False)
    print(f"Saved splits → agribot_train.csv ({len(train_df)}) | agribot_val.csv ({len(val_df)})")

print("Train/Val shapes:", train_df.shape, val_df.shape)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import shutil, os
os.makedirs("/content/drive/MyDrive/AgriBot", exist_ok=True)
for fname in ["agribot_dataset.csv","agribot_train.csv","agribot_val.csv"]:
    if os.path.exists(fname):
        shutil.copy(fname, f"/content/drive/MyDrive/AgriBot/{fname}")
print("Copied files to /MyDrive/AgriBot/")

**Reload your saved splits**

In [ ]:
import pandas as pd, os

if os.path.exists("agribot_train.csv") and os.path.exists("agribot_val.csv"):
    train_df = pd.read_csv("agribot_train.csv")
    val_df   = pd.read_csv("agribot_val.csv")
else:
    raise FileNotFoundError("Missing agribot_train.csv / agribot_val.csv. Re-run the split-save step.")

print(train_df.shape, val_df.shape)
train_df.head(2)

**Recreating tokenizer and model**

In [ ]:
!pip install -q "transformers==4.44.2" "tensorflow==2.18.0" "safetensors==0.4.4"

from transformers import (
    T5TokenizerFast, TFT5ForConditionalGeneration,
    AutoTokenizer, TFAutoModelForSeq2SeqLM
)

def load_t5_tf_with_fallback():
    # 1) Try native TF weights (t5-small)
    MODEL_1 = "t5-small"
    try:
        tok = T5TokenizerFast.from_pretrained(MODEL_1, use_fast=True)
        mdl = TFT5ForConditionalGeneration.from_pretrained(MODEL_1)
        print(" Loaded TF weights for:", MODEL_1)
        return tok, mdl, MODEL_1
    except Exception as e1:
        print(" Native TF load failed for t5-small:", type(e1).__name__, "-", e1)

    # 2) Try converting from PyTorch weights (t5-small)
    try:
        tok = T5TokenizerFast.from_pretrained(MODEL_1, use_fast=True)
        mdl = TFT5ForConditionalGeneration.from_pretrained(MODEL_1, from_pt=True)
        print(" Loaded from PT weights for:", MODEL_1)
        return tok, mdl, MODEL_1
    except Exception as e2:
        print(" from_pt load failed for t5-small:", type(e2).__name__, "-", e2)

    # 3) Fallback to FLAN-T5-small (TF auto model)
    MODEL_2 = "google/flan-t5-small"
    try:
        tok = AutoTokenizer.from_pretrained(MODEL_2, use_fast=True)
        mdl = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_2, from_pt=True)
        print(" Fallback loaded:", MODEL_2)
        return tok, mdl, MODEL_2
    except Exception as e3:
        raise RuntimeError(f"All loading attempts failed: {type(e3).__name__} - {e3}")

tokenizer, model, MODEL_NAME = load_t5_tf_with_fallback()
print("Using model:", MODEL_NAME)

**Rebuild datasets (cast to int32)**

In [ ]:
import numpy as np, tensorflow as tf

MAX_IN, MAX_OUT = 128, 128

def to_source(q: str) -> str:
    return f"question: {q} domain: agriculture"

def encode_pair_batch(questions, answers):
    enc = tokenizer(
        [to_source(q) for q in questions],
        max_length=MAX_IN, truncation=True, padding="max_length",
        return_tensors="np"
    )
    lbl = tokenizer(
        list(answers),
        max_length=MAX_OUT, truncation=True, padding="max_length",
        return_tensors="np"
    )["input_ids"]
    pad_id = tokenizer.pad_token_id
    lbl = np.where(lbl == pad_id, -100, lbl)
    # Cast to int32 for TF
    enc = {
        "input_ids":      enc["input_ids"].astype("int32"),
        "attention_mask": enc["attention_mask"].astype("int32"),
    }
    lbl = lbl.astype("int32")
    return enc, lbl

train_enc, train_lbl = encode_pair_batch(train_df["question"], train_df["answer"])
val_enc,   val_lbl   = encode_pair_batch(val_df["question"],   val_df["answer"])

BATCH_SIZE = 8  # if memory is tight, set to 4

train_ds = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_enc["input_ids"], "attention_mask": train_enc["attention_mask"]},
    train_lbl
)).shuffle(len(train_df), seed=42).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((
    {"input_ids": val_enc["input_ids"], "attention_mask": val_enc["attention_mask"]},
    val_lbl
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("train shapes:", train_enc["input_ids"].shape, train_lbl.shape)
print("val shapes:",   val_enc["input_ids"].shape,   val_lbl.shape)

**Manual baseline (no compile())**

In [ ]:
import math, tensorflow as tf

def eval_loss(ds):
    losses = []
    for (x, y) in ds:
        # Forward pass with labels; HF returns .loss
        out = model(input_ids=x["input_ids"], attention_mask=x["attention_mask"], labels=y, training=False)
        losses.append(float(out.loss.numpy()))
    return sum(losses) / max(1, len(losses))

base_loss = eval_loss(val_ds)
base_ppl  = math.exp(base_loss)
print(f"Manual baseline — val_loss: {base_loss:.4f} | perplexity: {base_ppl:.2f}")

**Manual training loop (Early Stopping + checkpoint)**

In [ ]:
# Optimizer from tf_keras (Keras 3) — safe for manual tape training
from tf_keras.optimizers import Adam
opt = Adam(learning_rate=3e-4, global_clipnorm=1.0)

EPOCHS = 12
PATIENCE = 2
best_val = float("inf")
wait = 0

train_hist, val_hist = [], []

ckpt_path = "t5_agribot_best.weights.h5"  # lightweight weights file

for epoch in range(1, EPOCHS+1):
    # ---- Train ----
    train_losses = []
    for (x, y) in train_ds:
        with tf.GradientTape() as tape:
            out = model(input_ids=x["input_ids"], attention_mask=x["attention_mask"], labels=y, training=True)
            loss = out.loss  # scalar
        grads = tape.gradient(loss, model.trainable_variables)
        opt.apply_gradients(list(zip(grads, model.trainable_variables)))
        train_losses.append(float(loss.numpy()))
    train_loss = sum(train_losses)/max(1,len(train_losses))

    # ---- Validate ----
    val_loss = eval_loss(val_ds)

    train_hist.append(train_loss); val_hist.append(val_loss)
    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f}")

    # ---- Early Stopping ----
    if val_loss + 1e-6 < best_val:
        best_val = val_loss
        wait = 0
        model.save_weights(ckpt_path)
        print(f"  ✓ improved. Saved weights → {ckpt_path}")
    else:
        wait += 1
        print(f"  no improvement ({wait}/{PATIENCE})")
        if wait >= PATIENCE:
            print("  Early stopping.")
            break

# Load best weights back
model.load_weights(ckpt_path)
print("Loaded best weights for evaluation.")

**Evaluate + BLEU + training curve (+ download)**

In [ ]:
# Final val metrics
final_loss = eval_loss(val_ds)
final_ppl  = math.exp(final_loss)
print(f"After fine-tuning — val_loss: {final_loss:.4f} | perplexity: {final_ppl:.2f}")

In [ ]:
# Install if missing (safe to run multiple times)
import sys, subprocess, pkgutil
if not pkgutil.find_loader("sacrebleu"):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "sacrebleu==2.4.0"])

import sacrebleu

def generate_text(q):
    inp = tokenizer(to_source(q), return_tensors="tf", truncation=True, max_length=128)
    out_ids = model.generate(**inp, max_new_tokens=120, temperature=0.7, top_p=0.95)
    return tokenizer.decode(out_ids[0], skip_special_tokens=True)

# Build refs & hyps for the validation split
hyps, refs = [], []
for _, row in val_df.iterrows():
    hyps.append(generate_text(row["question"]))
    refs.append(row["answer"])

bleu = sacrebleu.corpus_bleu(hyps, [refs])  # sacrebleu expects list-of-systems vs list-of-references
print(f"BLEU: {bleu.score:.2f}")

**Save a CSV of questions, references, and hypotheses (download-ready)**

In [ ]:
import pandas as pd

eval_rows = []
for i, (_, row) in enumerate(val_df.iterrows()):
    eval_rows.append({
        "question": row["question"],
        "reference_answer": row["answer"],
        "model_answer": hyps[i],
        "bucket": row.get("bucket", None)
    })

eval_df = pd.DataFrame(eval_rows)
eval_df.to_csv("agribot_eval_val_predictions.csv", index=False)
print("Saved: agribot_eval_val_predictions.csv")

# (Colab only) uncomment to download:
# from google.colab import files
# files.download("agribot_eval_val_predictions.csv")

**BLEU by bucket chart**

In [ ]:
# Sentence-level BLEU then average per bucket (sacrebleu supports sentence_bleu)
import numpy as np
import matplotlib.pyplot as plt

sent_bleus = []
for ref, hyp in zip(refs, hyps):
    sb = sacrebleu.sentence_bleu(hyp, [ref]).score
    sent_bleus.append(sb)

eval_df["sentence_bleu"] = sent_bleus

if "bucket" in eval_df.columns and eval_df["bucket"].notna().any():
    bucket_bleu = eval_df.groupby("bucket")["sentence_bleu"].mean().sort_values(ascending=False)
    ax = bucket_bleu.plot(kind="bar", figsize=(8,4))
    ax.set_title("AgriBot — Mean Sentence BLEU by Bucket (Validation)")
    ax.set_xlabel("Bucket"); ax.set_ylabel("Mean Sentence BLEU")
    plt.xticks(rotation=45, ha="right"); plt.tight_layout()
    plt.savefig("bleu_by_bucket.png", dpi=200)
    print("Saved: bleu_by_bucket.png")

    # Colab download (optional)
    # from google.colab import files
    # files.download("bleu_by_bucket.png")
else:
    print("No 'bucket' column found; skipping BLEU-by-bucket chart.")

**Token-level F1**

In [ ]:
import re
def normalize(s):
    return re.sub(r"\s+", " ", s.strip().lower())

def f1_token_level(pred, gold):
    p = normalize(pred).split()
    g = normalize(gold).split()
    common = set(p) & set(g)
    if len(p) == 0 or len(g) == 0:
        return 0.0
    prec = len(common) / len(p)
    rec  = len(common) / len(g)
    if prec + rec == 0:
        return 0.0
    return 2 * prec * rec / (prec + rec)

eval_df["f1_token"] = [f1_token_level(h, r) for h, r in zip(hyps, refs)]
print("Mean token-level F1 on validation:", float(eval_df["f1_token"].mean()))
eval_df.to_csv("agribot_eval_val_predictions.csv", index=False)  # overwrite with F1 column

**deterministic BLEU + F1 (no randomness)**

In [ ]:
# Ensure sacrebleu is installed
import sys, subprocess, pkgutil
if not pkgutil.find_loader("sacrebleu"):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "sacrebleu==2.4.0"])
import sacrebleu, math, pandas as pd

def generate_text_beam(q, beams=4, max_new=120):
    inp = tokenizer(to_source(q), return_tensors="tf", truncation=True, max_length=128)
    out_ids = model.generate(**inp, num_beams=beams, do_sample=False, max_new_tokens=max_new)
    return tokenizer.decode(out_ids[0], skip_special_tokens=True)

# Recompute metrics deterministically
hyps_det, refs = [], []
for _, row in val_df.iterrows():
    hyps_det.append(generate_text_beam(row["question"]))
    refs.append(row["answer"])

bleu_det = sacrebleu.corpus_bleu(hyps_det, [refs]).score
print(f"Deterministic BLEU (beams=4): {bleu_det:.2f}")

import re
def normalize(s): return re.sub(r"\s+", " ", s.strip().lower())
def f1_token(pred, gold):
    p, g = normalize(pred).split(), normalize(gold).split()
    if not p or not g: return 0.0
    inter = set(p) & set(g)
    if not inter: return 0.0
    prec, rec = len(inter)/len(p), len(inter)/len(g)
    return 0.0 if (prec+rec)==0 else 2*prec*rec/(prec+rec)

mean_f1_det = sum(f1_token(h, r) for h, r in zip(hyps_det, refs)) / len(refs)
print(f"Deterministic token F1: {mean_f1_det:.3f}")

**running an experiment end-to-end (manual loop)**

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from tf_keras.optimizers import Adam

def run_experiment(
    exp_name="expA_t5small_lr3e4",
    model_name="t5-small",
    lr=3e-4,
    epochs=12,
    patience=2,
    beams=4,
    batch_size=8,
    max_in=128,
    max_out=128
):
    print(f"\n===== {exp_name} :: {model_name} | lr={lr} | epochs={epochs} | bs={batch_size} =====")
    # Load tokenizer+model (auto TF, from_pt for safety)
    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    mdl = TFAutoModelForSeq2SeqLM.from_pretrained(model_name, from_pt=True)

    # Build fresh datasets for this tokenizer (pad ids can differ slightly)
    def to_src(q): return f"question: {q} domain: agriculture"
    def encode_df(qs, ans):
        enc = tok([to_src(q) for q in qs], max_length=max_in, truncation=True, padding="max_length", return_tensors="np")
        lbl = tok(list(ans), max_length=max_out, truncation=True, padding="max_length", return_tensors="np")["input_ids"]
        pad_id = tok.pad_token_id
        import numpy as np
        lbl = np.where(lbl == pad_id, -100, lbl).astype("int32")
        enc = { "input_ids": enc["input_ids"].astype("int32"),
                "attention_mask": enc["attention_mask"].astype("int32") }
        return enc, lbl

    tr_enc, tr_lbl = encode_df(train_df["question"], train_df["answer"])
    va_enc, va_lbl = encode_df(val_df["question"],   val_df["answer"])

    tr_ds = tf.data.Dataset.from_tensor_slices(({"input_ids": tr_enc["input_ids"], "attention_mask": tr_enc["attention_mask"]}, tr_lbl)).shuffle(len(train_df), seed=42).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    va_ds = tf.data.Dataset.from_tensor_slices(({"input_ids": va_enc["input_ids"], "attention_mask": va_enc["attention_mask"]}, va_lbl)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    # Eval function (no compile)
    import math
    def eval_loss(ds):
        losses=[]
        for x,y in ds:
            out = mdl(input_ids=x["input_ids"], attention_mask=x["attention_mask"], labels=y, training=False)
            losses.append(float(out.loss.numpy()))
        return sum(losses)/max(1,len(losses)), lambda v: math.exp(v)

    base_loss, exp_fn = eval_loss(va_ds)
    base_ppl = exp_fn(base_loss)
    print(f"Baseline → val_loss={base_loss:.4f} | ppl={base_ppl:.2f}")

    # Manual training loop
    opt = Adam(learning_rate=lr, global_clipnorm=1.0)
    best = float("inf"); wait = 0
    train_hist, val_hist = [], []
    ckpt = f"{exp_name}.weights.h5"

    for ep in range(1, epochs+1):
        # train
        train_losses=[]
        for x,y in tr_ds:
            with tf.GradientTape() as tape:
                out = mdl(input_ids=x["input_ids"], attention_mask=x["attention_mask"], labels=y, training=True)
                loss = out.loss
            grads = tape.gradient(loss, mdl.trainable_variables)
            opt.apply_gradients(zip(grads, mdl.trainable_variables))
            train_losses.append(float(loss.numpy()))
        tr_loss = sum(train_losses)/max(1,len(train_losses))

        # val
        va_loss, _ = eval_loss(va_ds)
        train_hist.append(tr_loss); val_hist.append(va_loss)
        print(f"Epoch {ep:02d} | train={tr_loss:.4f} | val={va_loss:.4f}")

        if va_loss + 1e-6 < best:
            best = va_loss; wait = 0
            mdl.save_weights(ckpt)
            print(f"  ✓ improved → saved {ckpt}")
        else:
            wait += 1
            print(f"  no improve ({wait}/{patience})")
            if wait >= patience:
                print("  Early stopping.")
                break

    # Load best & final eval
    mdl.load_weights(ckpt)
    final_loss, _ = eval_loss(va_ds)
    final_ppl = exp_fn(final_loss)

    # Deterministic BLEU + F1
    import sacrebleu, re
    def norm(s): return re.sub(r"\s+", " ", s.strip().lower())
    def f1_token(pred, gold):
        p, g = norm(pred).split(), norm(gold).split()
        if not p or not g: return 0.0
        inter = set(p) & set(g)
        if not inter: return 0.0
        prec, rec = len(inter)/len(p), len(inter)/len(g)
        return 0.0 if (prec+rec)==0 else 2*prec*rec/(prec+rec)

    def gen_beam(q):
        ids = tok(to_src(q), return_tensors="tf", truncation=True, max_length=max_in)
        out = mdl.generate(**ids, num_beams=beams, do_sample=False, max_new_tokens=120)
        return tok.decode(out[0], skip_special_tokens=True)

    hyps, refs = [], []
    for _, row in val_df.iterrows():
        hyp = gen_beam(row["question"])
        hyps.append(hyp)
        refs.append(row["answer"])

    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    f1   = sum(f1_token(h,r) for h,r in zip(hyps, refs))/len(refs)

    # Save per-exp artifacts
    import matplotlib.pyplot as plt
    plt.figure(figsize=(6,4))
    plt.plot(train_hist, label="train"); plt.plot(val_hist, label="val")
    plt.title(f"{exp_name} — Loss"); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.tight_layout()
    curve_png = f"{exp_name}_curve.png"
    plt.savefig(curve_png, dpi=200)

    # Return a summary row
    return {
        "exp": exp_name,
        "model": model_name,
        "lr": lr,
        "epochs": epochs,
        "batch": batch_size,
        "baseline_ppl": round(base_ppl, 3),
        "final_ppl":    round(final_ppl, 3),
        "bleu":         round(bleu, 2),
        "f1_token":     round(f1, 3),
        "curve_image":  curve_png,
        "ckpt":         ckpt
    }

**Running two experiments**

In [ ]:
results = []

# B) Lower LR (often more stable on tiny datasets)
results.append(run_experiment(
    exp_name="expB_t5small_lr1e4",
    model_name="t5-small",
    lr=1e-4,
    epochs=15,
    patience=3,
    beams=4,
    batch_size=8
))

# C) Instruction-tuned model (often better: FLAN-T5)
results.append(run_experiment(
    exp_name="expC_flan_t5small_lr2e4",
    model_name="google/flan-t5-small",
    lr=2e-4,
    epochs=12,
    patience=3,
    beams=4,
    batch_size=8
))

import pandas as pd
res_df = pd.DataFrame(results)
display(res_df)
res_df.to_csv("agribot_experiments.csv", index=False)
print("Saved: agribot_experiments.csv")